In [ ]:
import dotenv
import logging
import json
import os

import pathway as pw
from pathway.xpacks.llm import embedders, parsers, splitters, vector_store

In [ ]:
logger = logging.getLogger(__name__)

In [ ]:
config = dotenv.dotenv_values('.env')
config

In [ ]:
protocol_name = 'aave'
prop_order_by = 'asc'
api_key = config['BOARDROOM_API_KEY']

## Protocol

In [ ]:
class Protocol(pw.Schema):
    cname: str
    name: str
    categories: str
    is_enabled: bool
    active_on_website: bool
    total_proposals: int
    total_votes: int
    unique_voters: int
    # tokens: list[object]
    ptype: str
    # delegated_support: dict


In [ ]:
def protocol_mapper(raw_data: bytes) -> bytes:
    # logger.info(raw_data.decode())
    data = json.loads(raw_data.decode())["data"]
    return json.dumps(
        {
            "cname": data["cname"],
            "name": data["name"],
            "categories": ",".join(data["categories"]),
            "is_enabled": data["isEnabled"],
            "active_on_website": data["activeOnWebsite"],
            "total_proposals": data["totalProposals"],
            "total_votes": data["totalVotes"],
            "unique_voters": data["uniqueVoters"],
            "ptype": data["type"],
        }
    ).encode()


In [ ]:

protocol = pw.io.http.read(
    f"https://api.boardroom.info/v1/protocols/{protocol_name}?key={api_key}",
    method='GET',
    headers={"Accept": "application/json"},
    # format="raw",
    schema=Protocol,
    response_mapper=protocol_mapper
)

In [ ]:
protocol.schema

In [ ]:
protocol

## Proposals

In [ ]:
# class Proposal(pw.Schema):
#     protocol: str
#     ref_id: str
#     pid: str
#     title: str
#     content: str
#     adapter: str
#     proposer: str
#     total_votes: int
#     block_number: int
#     external_url: str
#     start_timestamp: int
#     end_timestamp: int
#     current_state: str
#     # results: list[dict] # TODO
#     ptype: str
#     summary: str
#     privacy: str
#     # executables: dict
#     tx_hash: str
#     quorum: int



In [ ]:
# def coalesce(data: dict, key: str, fallback: str = ''):
#     return data[key] if key in data else fallback

# def proposal_mapper(raw_data: bytes) -> bytes:
#     json_data = json.loads(raw_data.decode())
#     data_records = json_data["data"]
#     next_cursor = json_data["nextCursor"]
#     # return json.dumps([
#     #     {
#     #         "protocol": coalesce(data, "protocol"),
#     #         "ref_id": coalesce(data, "refId"),
#     #         "pid": coalesce(data, "id"),
#     #         "title": coalesce(data, "title"),
#     #         "content": coalesce(data, "content"),
#     #         "adapter": coalesce(data, "adapter"),
#     #         "proposer": coalesce(data, "proposer"),
#     #         "total_votes": coalesce(data, "totalVotes"),
#     #         "block_number": coalesce(data, "blockNumber"),
#     #         "external_url": coalesce(data, "externalUrl"),
#     #         "start_timestamp": int(coalesce(data, "startTimestamp")),
#     #         "end_timestamp": int(coalesce(data, "endTimestamp")),
#     #         "current_state": coalesce(data, "currentState"),
#     #         # "results": [{"choice": data["choices"][choice["choice"]], "result": choice["total"]} for choice in data["results"]],
#     #         "ptype": coalesce(data, "type"),
#     #         "summary": coalesce(data, "summary"),
#     #         "privacy": coalesce(data, "privacy"),
#     #         "tx_hash": coalesce(data, "txHash"),
#     #         "quorum": coalesce(data, "quorum"),
#     #     } for data in data_records
#     data = data_records[0]
#     return json.dumps(
#         {
#             "protocol": coalesce(data, "protocol"),
#             "ref_id": coalesce(data, "refId"),
#             "pid": coalesce(data, "id"),
#             "title": coalesce(data, "title"),
#             "content": coalesce(data, "content"),
#             "adapter": coalesce(data, "adapter"),
#             "proposer": coalesce(data, "proposer"),
#             "total_votes": coalesce(data, "totalVotes"),
#             "block_number": coalesce(data, "blockNumber"),
#             "external_url": coalesce(data, "externalUrl"),
#             "start_timestamp": int(coalesce(data, "startTimestamp")),
#             "end_timestamp": int(coalesce(data, "endTimestamp")),
#             "current_state": coalesce(data, "currentState"),
#             # "results": [{"choice": data["choices"][choice["choice"]], "result": choice["total"]} for choice in data["results"]],
#             "ptype": coalesce(data, "type"),
#             "summary": coalesce(data, "summary"),
#             "privacy": coalesce(data, "privacy"),
#             "tx_hash": coalesce(data, "txHash"),
#             "quorum": coalesce(data, "quorum"),
#         }
#     ).encode()

In [ ]:
class BoardroomAPI(pw.Schema):
    data: pw.Json

In [ ]:
proposals = pw.io.http.read(
    f"https://api.boardroom.info/v1/protocols/{protocol_name}/proposals?key={api_key}&orderByIndexedAt{prop_order_by}",
    method='GET',
    headers={"Accept": "application/json"},
    format="json",
    schema=BoardroomAPI
    # schema=Proposal,
    # response_mapper=proposal_mapper
)

In [ ]:
# u_logger = logging.getLogger("unstructured")
# u_logger.setLevel(logging.INFO)

In [ ]:
# export CFLAGS="-Wno-nullability-completeness" if trying to install pillow-heif missingn module
# libmagic -> Required for having libmagic working:
# - brew install libmagic
# - pip install python-magic-bin
parser = parsers.ParseUnstructured(mode="elements")

# @pw.udf(executor=pw.udfs.async_executor())
# def parse_document(raw_text: pw.Json) -> list[tuple[str, dict]]:
#     return parser(f"{raw_text}".encode())

In [ ]:
proposals = proposals.flatten(proposals.data)
proposals = proposals.with_columns(
    title=pw.this.data.get("title", default=pw.Json("Untitled")).as_str(),
    # content=pw.apply_with_type(lambda x: f"{x}".encode() if x else b"", bytes, pw.this.data.get("content", default=None)),
    content=parser(pw.apply_with_type(lambda x: f"{x}".encode() if x else b"", bytes, pw.this.data.get("content", default=None))),
    # content=pw.this.data.get("content", default=None)
    # parsed_content=parser(pw.this.data.get("content", default=pw.Json("Empty"))),
)
proposals = proposals.flatten(pw.this.content)
proposals = proposals.select(title=pw.this.title, text=pw.this.content[0], metadata=pw.this.content[1])


In [ ]:
proposals.schema

In [ ]:
proposals

## Run workflow

In [ ]:
# pw.debug.compute_and_print(proposals.select(content=pw.this.content))

In [ ]:
%%capture --no-display
pw.run()

In [ ]:
test_doc = """
---\n\ntitle: ARC - Strategic Partnership with Balancer Part #2\nstatus: Discussion\nauthor: Llama Community - @MatthewGraham\ncreated: 03-05-2022\n\n---\n\n# Simple Summary\n\nAcquire 100,000 BAL tokens by deploying a Bonding Curve contract. The BAL tokens are to be held in the Reserve Factor until further details relating to Aura emerges and then the Aave community can determine if to deploy BAL to receive veBAL or auraBAL, [1].\n\n# Abstract\n\nDeploy a Bonding Curve to acquire an additional 100,000 BAL tokens on market at the chainlink oracle price +50 bps, for a cost $1,708,500 assuming a BAL price of $17. This is an additional purchase to the 200,000 BAL token swap being considered.\n\nAave will become one of eight token holders with 300,000 BAL tokens. All 100,000 BAL tokens are to be retained in the Reserve Factor until such time as further information on auraBAL emerges.\n\n# Motivation\n\nThe original motivation for performing a token swap with Balancer can be found on the ARC - Strategic Investment BAL Token, [2].\n\nWith the additional 100,000 BAL acquisition, Aave would transition from just outside the Top 50 BAL holder to inside the Top 30 holders, [3]. If an additional 75,000 BAL was acquired, totalling 375,000 BAL, then Aave would become a Top 20 holder and this includes three community wallets, one Polygon bridge pool and the main BAL liquidity pool. So really it would be more like the Top 15 holders if another 75,000 BAL was acquired.\n\nWith 300,000 BAL tokens, Aave would have the largest BAL holding of any strategic partner Balancer has partnered with. The largest strategic partnership holding to date is the 270,000 BAL position held by Tribe DAO, [4,5]. Having a larger BAL holding relative to Tribe DAO may become a competitive advantage as Tribe DAO seeks to direct TVL to Rari Fuse pools with the use of the new ERC-4626 vault, [6].\n\nWith the emergence of ERC-4626, a new type of vault, whereby Liquidity Providers (LP) can deposit their stakes BPT tokens into lending pools and earn BAL rewards whilst using staked BPT as collateral, [7]. This is particularly appealing for staked BPT holdings which have minimal Impermanent loss risk. An example would be to use the staked BPT token for the stable coin Balancer Boosted Pool which uses Aave V2 to generate yield and receives BAL rewards, [8]. Based upon the success of wstETH being listed on Aave V2, this may lead to significant Total Value Locked (TVL) growth for Aave which makes up more than 13% of Aave’s mainnet V2 market TVL, [9].\n\nHaving not yet entered in the Curve / Convex tokenomics play, Aave has an opportunity to be an early leader in the veBAL wars. With the emergence of Aura, the Aave community has a choice to determine if the 100,000 BAL to be acquired are to be deployed for veBAL or auraBAL. If Aura is like Convex, then being early and building an Aura holding is worth considering. The first step in both strategies is to acquire a sizable BAL position and as the initial 200,000 BAL is proposed to be deposited into the veBAL contract and locked for 1 year, the 100,000 BAL detailed within this proposal can be deployed into either veBAL or auraBAL contracts.\n\nThere are likely to be many more Balancer Boosted Pools that utilise aTokens in the future and the veBAL votes can be used to help bootstrap these pools by directing BAL rewards to those pools. Using either veBAL or auraBAL to vote for BAL rewards towards pools that create TVL for Aave is a strategic advantage. If Aura evolves like Convex did, then the voter locked AURA token will have more voting influence over BAL rewards than one veBAL token. Being prepared and ready for the Aura token launch is going to position Aave to maximise the upside from its investments as rewards often favour early movers.\n\n# Implementation\n\nA Bonding Curve deployment utilising Chainlink’s BAL/USD oracle feed with a 50 bps incentive for arbitrage trades to acquire BAL on market and deposit into the contract. Aave will provide USDC to fund the Bonding Curve contract. An example of this type of transaction occurred when Tribe purchased $10M of DPI on market, [10].\n\nThe 50 bps fee is to provide an economic incentive for arbitrage traders to attain BAL on market and deposit BAL into the contract profitably by more than offsetting gas costs. This mechanism enables everyone within defi to participate and essentially crowdsources the BAL tokens on Aave’s behalf. Once the BAL has been received it will be transferred to the Reserve Factor.\n\n# References\n\n[1] https://forum.balancer.fi/t/proposal-allowlist-aura-finance-in-balancer-votingescrow/2708\n[2] https://governance.aave.com/t/arc-strategic-partnership-with-balancer/7617/15\n[3] https://etherscan.io/token/0xba100000625a3754423978a60c9317c58a424e3d#balances\n[4] https://forum.balancer.fi/t/proposal-allow-tribe-dao-to-lock-vebal/2888\n[5] https://snapshot.org/#/balancer.eth/proposal/0x7972e1700ac30495ad936038b66055b1e6420c3de5ed1fec963abe5faa2d16a3\n[6] https://tribe.fei.money/t/fip-90-fuse-boosted-usd-balancer-pool-bb-f-usd/4023\n[7] https://eips.ethereum.org/EIPS/eip-4626\n[8] https://app.balancer.fi/#/pool/0x7b50775383d3d6f0215a8f290f2c9e2eebbeceb20000000000000000000000fe\n[9] https://app.aave.com/reserve-overview/?underlyingAsset=0xae7ab96520de3a18e5e111b5eaab095312d7fe84&marketName=proto_mainnet\n[10] https://etherscan.io/address/0x464c71f6c2f760dda6093dcb91c24c39e5d6e18c\n\n# Copyright\n\nCopyright and related rights waived via [CC0](https://creativecommons.org/publicdomain/zero/1.0/).
"""

In [ ]:
tt = (('"## Simple Summary\\n\\nA proposal to adjust sAVAX\\u2019s LT and LTV on Aave V3 Avalanche.\\n\\n## Motivation\\n\\nIncreasing the liquidation threshold and LTV for assets allows Aave to enhance users\\u2019 capital efficiency. However, this must be balanced with proper risk management to ensure that there is a sufficient buffer in the event of large drawdowns and/or liquidations. The analysis below was conducted utilizing our LT simulations, which showed either minimal or no increases in VaR at the recommended LT levels while also considering user distribution and on-chain liquidity.\\n\\n### sAVAX\\n\\nsAVAX\\u2019s LTV and LT are currently set to 30% and 40%, respectively (its E-Mode parameters are 92.5% and 95%).',
       pw.Json({'category': 'NarrativeText', 'filetype': 'text/plain', 'languages': ['eng']})),
      ('The top suppliers are primarily looping sAVAX and WAVAX, reducing the risk of large-scale liquidations in this market.\\nHowever, there is a small amount of non-WAVAX borrows against sAVAX, primarily WETH.e and stablecoins.\\nMoreover, sAVAX maintains strong liquidity against AVAX, meaning that potential liquidations of sAVAX collateralized debt are likely to be completed efficiently.\\nGiven user distribution and liquidity, we recommend increasing LTV to 40% and LT to 45%.\\n\\n## Specification\\n\\nGiven these observations, we recommend making the following changes:\\n\\n| Chain     | Asset | Parameter             | Current | Recommended |\\n| --------- | ----- | --------------------- | ------- | ----------- |\\n| Avalanche | sAVAX | LTV                   | 30%     | 40%         |\\n| Avalanche | sAVAX | Liquidation Threshold | 40%     | 45%         |\\n\\n## References\\n\\n- Implementation:',
       pw.Json({'category': 'NarrativeText', 'filetype': 'text/plain', 'languages': ['eng']})),
      ('[AaveV3Avalanche](https://github.com/bgd-labs/aave-proposals-v3/blob/1391ea39052a0d35fbd843bd49edaf6ea7eb1c73/src/20240724_AaveV3Avalanche_RiskParameterUpdatesSAVAXOnAaveV3Avalanche/AaveV3Avalanche_RiskParameterUpdatesSAVAXOnAaveV3Avalanche_20240724.sol)\\n- Tests: [AaveV3Avalanche](https://github.com/bgd-labs/aave-proposals-v3/blob/1391ea39052a0d35fbd843bd49edaf6ea7eb1c73/src/20240724_AaveV3Avalanche_RiskParameterUpdatesSAVAXOnAaveV3Avalanche/AaveV3Avalanche_RiskParameterUpdatesSAVAXOnAaveV3Avalanche_20240724.t.sol)\\n- [Snapshot](https://snapshot.org/#/aave.eth/proposal/0x471ab55b0091043963c744f228befd842aeb354b0d04c76da3c9eb2b401934a4)\\n- [Discussion](https://governance.aave.com/t/arfc-chaos-labs-risk-parameter-updates-savax-on-aave-v3-avalanche-07-16-2024/18277)\\n\\n## Copyright\\n\\nCopyright and related rights waived via [CC0](https://creativecommons.org/publicdomain/zero/1.0/)."', 
       pw.Json({'category': 'NarrativeText', 'filetype': 'text/plain', 'languages': ['eng']})))

In [ ]:
(text, parsed_content) = tt